In [1]:
%load_ext watermark
%watermark -a 'Sebastian Raschka' -v -d -p biopython

# Fetch Iridium Dataset

In [2]:
iridium_ht_codes = """
1A28, 1AI5, 1AZM, 1B9V, 1BR6, 1C1B, 1CTR, 
1CVU, 1CX2, 1D3H, 1DD7, 1DDS, 1EOC, 1EXA, 
1EZQ, 1F0S, 1F0T, 1F0U, 1FCX, 1FCZ, 1FH8, 
1FH9, 1FHD, 1FJS, 1FL3, 1FM6, 1FM9, 1FQ5, 
1FRP, 1FVT, 1G9V, 1GM8, 1GWX, 1H1P, 1H1S, 
1HDY, 1HGG, 1HGH, 1HGI, 1HGJ, 1HNN, 1HP0, 
1HQ2, 1HVY, 1HWI, 1HWW, 1IVB, 1IVD, 1IVE, 
1IVF, 1IY7, 1JD0, 1JLA, 1K1J, 1K3U, 1KE5, 
1L2S, 1L7F, 1LPZ, 1LQD, 1LRH, 1M2Z, 1MBI, 
1ML1, 1MMV, 1MQ6, 1MTS, 1MZC, 1N1M, 1N2J, 
1N2V, 1NAV, 1OF1, 1OF6, 1OQ5, 1OWE, 1OYT, 
1P2Y, 1P62, 1PBD, 1PMN, 1PSO, 1Q1G, 1Q41, 
1QHI, 1R58, 1R9O, 1ROB, 1S19, 1SQ5, 1TOW, 
1TT1, 1U4D, 1UKZ, 1ULB, 1UML, 1UNL, 1UOU, 
1V0P, 1W1P, 1W2G, 1X8X, 1XM6, 1XOQ, 1YDR, 
1YDS, 1YDT, 1YQY, 1YV3, 1YVF, 1YWR, 2ACK, 
2BR1, 2CTC, 2MCP, 2PCP, 2TMN, 3PTB, 4AAH, 
4COX, 4TS1"""

iridium = [i.strip() for i in iridium_ht_codes.split(', ')]

In [3]:
import urllib.request
import os

def fetch_pdb(pdb_code):   
    txt = None
    try:
        response = urllib.request.urlopen('http://www.rcsb.org/pdb/files/%s.pdb' % pdb_code.lower())
        txt = response.read().decode('utf-8')
    except urllib.request.HTTPError as e:
        print('HTTP Error %s' %e.code)
    except urllib.request.URLError as e:
        print('URL Error %s' %e.args) 
    return txt

for i in iridium:
    i = i.lower()
    txt = fetch_pdb(i)
    with open('./iridium_ht/%s.pdb' % i, 'w') as f:
        f.write(txt)

URL Error [Errno 60] Operation timed out


TypeError: write() argument must be str, not None

# Benchmarking biopython

In [ ]:
codes = [i for i in os.listdir('./iridium_ht/') if i.endswith('.pdb')]
paths = [os.path.join('./iridium_ht/', i) for i in codes]
codes = [i.strip('.pdb') for i in codes]

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import os
import timeit
from Bio.PDB.PDBParser import PDBParser

def biopython_getresidues(pdbcode, path):
    parser = PDBParser()
    structure = parser.get_structure(pdbcode, path)
    res = []
    for residue in structure.get_residues():
        res.append(residue)
    
def f_test():
    biopython_getatoms(c, p)

bpython_times = []
for c, p in zip(codes, paths):
    t = timeit.timeit("f_test()", setup="from __main__ import f_test", number=10)
    bpython_times.append((t/10))

# Benchmarking biopandas

In [ ]:
import os
import timeit
from biopandas.pdb impport PandasPDB

def biopandas_getatoms(path):
    ppdb = PandasPDB().read_pdb(path)
    residues = ppdb.df['ATOM']['resi_name']
    
def f_test():
    biopandas_getatoms(p)

bpandas_times = []
for p in paths:
    t = timeit.timeit("f_test()", setup="from __main__ import f_test", number=10)
    bpandas_times.append((t/10))

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
with plt.style.context('fivethirtyeight'):
    fig = plt.figure(figsize=(29,4))

    index = np.arange(n_groups)
    bar_width = 0.2
    opacity = 0.3

    x = np.array(range(len(codes)))
    rects1 = plt.bar(x, bpandas_times, bar_width,
                     alpha=opacity,
                     color='b',
                     label='BioPandas')


    rects2 = plt.bar(x + bar_width, bpython_times, bar_width,
                     alpha=opacity,
                     color='r',
                     label='BioPython')


    plt.xticks(x + bar_width, codes, rotation=90)
    plt.legend()

    plt.tight_layout()
    plt.show()

In [ ]:
with plt.style.context('fivethirtyeight'):
    plt.plot(x, bpandas_times, alpha=0.5, marker='o', label='BioPandas')
    plt.plot(x, bpython_times, alpha=0.5, label='BioPython')
    plt.ylabel('Time to extract residues [sec]')
    plt.xlabel('PDBs from the Iridium Dataset')
    plt.legend(numpoints=1)
    plt.xticks([])
plt.tight_layout()
plt.show()

In [ ]:
with plt.style.context('fivethirtyeight'):
    fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(12, 5))

    # generate some random test data
    all_data = [bpython_times, bpandas_times]

    # plot violin plot
    axes.violinplot(all_data,
                       showmeans=False,
                       showmedians=True)
    #axes.set_title('Extracing Atom Names')


    # add x-tick labels
    plt.setp(axes, xticks=[y+1 for y in range(len(all_data))],
             xticklabels=['biopython', 'biopandas'])
    plt.show()